In [1]:
from enum import Enum
from typing import Union
import random

class MMGame:
    def __init__(self, code_length: int=4, secret_code: str=None, replace: bool=False):
        """ Initialize game with code length, optional secret, and replace mode """
        self.colours: dict[str, str] = {'R': 'Red', 'O':'Orange', 'Y':'Yellow',
                                       'G':'GREEN', 'B':'Blue', 'I':'Indigo'}
        self.code_length: int = code_length
        self.all_codes: list[str] = self._generate_all_codes(replace=replace)
        self.secret_code: str = secret_code if secret_code else random.choice(self.all_codes)

    def _generate_all_codes(self, replace: bool=False) -> list[str]:
        """ Generate all possible codes with or without replacement."""
        from itertools import permutations, product
        if replace:
            return [''.join(p) for p in product(self.colours.keys(), repeat=self.code_length)]
        else:
            return [''.join(p) for p in permutations(self.colours.keys(), self.code_length)]


In [2]:
from collections import Counter
  
class KnuthSolver:

    class Peg(Enum):
        BLACK = 2
        WHITE = 1
        NONE = 0
    
    def __init__(self, game: MMGame):
        """ Initialize solver with a MasterMind game with all possible codes."""
        self.game: MMGame = game
        self.possible_codes: list[str] = self.game.all_codes.copy()
        self.history = []

    def _get_feedback(self, guess: str, ref_code: str=None) -> list[Peg]:
        """ Compute feedback pegs comparing guess to reference code or secret """
        ref_code = ref_code if ref_code else self.game.secret_code
        black = sum(g == c for g, c in zip(guess, ref_code))
        white = sum(min(guess.count(c), ref_code.count(c))
                    for c in set(self.game.colours.keys())) - black
        feedback = [self.Peg.BLACK] * black + [self.Peg.WHITE] * white
        feedback += [self.Peg.NONE] * (self.game.code_length - len(feedback))
        return feedback
    
    def score_feedback(self, code: str, guess: str=None) -> tuple[Peg]:
        """ Return a sorted tuple of Peg feedback for a given code and guess """
        return tuple(
            sorted(
                self._get_feedback(code, guess),
                key=lambda x: x.value,
                reverse=True
            )
        )

    def prune_candidates(self, guess: str, feedback: tuple[Peg]):
        """ Prune possible_codes to those that would yield same feedback for guess """
        self.possible_codes = [
            c for c in self.possible_codes
            if self.score_feedback(c, guess) == feedback
        ]
    
    def next_guess(self) -> str:
        """ Select next guess by minimizing maximum partition size over remaining codes """
        minmax = {}
        for guess in self.possible_codes:
            score_counts = Counter(
                self.score_feedback(guess, ref_code)
                for ref_code in self.possible_codes
            )
            max_score = max(score_counts.values())
            minmax[guess] = max_score

        min_score = min(minmax.values())
        best_guesses = [g for g, score in minmax.items() if score == min_score]

        # heuristic: choose a candidate from possible_codes if available, else first guess
        return next((g for g in best_guesses if g in self.possible_codes), best_guesses[0])  

    def solve(self, initial_guess: str):
        """ Play Knuth's 5 move algorithm from initial_guess until the code is found """
        guess = initial_guess
        while True:
            feedback = self.score_feedback(guess)
            self.history.append((guess, feedback))
            if feedback.count(self.Peg.BLACK) == self.game.code_length:
                return guess

            self.prune_candidates(guess, feedback)
            guess = self.next_guess()

In [5]:
import timeit
_start = timeit.default_timer()

# mm_game = MMGame(secret_code='RPOY', replace=False)
mm_game = MMGame(replace=True, code_length=4, secret_code='YGOO') 
solver = KnuthSolver(mm_game)
print(f"Secret code: {mm_game.secret_code}")
solution = solver.solve(initial_guess='RGBY')
print(f"Game solved in {len(solver.history)} guesses!")
for guess, feedback in solver.history:
    print(f"Guess: {guess}, Feedback: {[f.name for f in feedback]}")
assert solution == mm_game.secret_code 
_stop = timeit.default_timer()
print(f"Elapsed time: {_stop - _start:.6f} seconds")

Secret code: YGOO
Game solved in 5 guesses!
Guess: RGBY, Feedback: ['BLACK', 'WHITE', 'NONE', 'NONE']
Guess: RROG, Feedback: ['BLACK', 'WHITE', 'NONE', 'NONE']
Guess: OIBG, Feedback: ['WHITE', 'WHITE', 'NONE', 'NONE']
Guess: ROYI, Feedback: ['WHITE', 'WHITE', 'NONE', 'NONE']
Guess: YGOO, Feedback: ['BLACK', 'BLACK', 'BLACK', 'BLACK']
Elapsed time: 0.333397 seconds
